___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [1]:
!pip install requests

In [2]:
import requests

In [3]:
r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=USD&date=20210815&json')
print('Підключення вдале', r)

Підключення вдале <Response [200]>


2. Отримати курс долара за останній рік.

In [4]:
import datetime

import requests


def currency_data(start_date, end_date):
    data_base = {}
    day = start_date

    while day != end_date:
        exchange_url = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?date='+day.strftime('%Y%m%d')+'&json?'
        r = requests.get(exchange_url)
        if r.status_code == 200:
            data = r.json()
            for currency in data:
                if currency["cc"] == "USD":
                    curr = currency["txt"]
                    rate = currency["rate"]
                    date = currency["exchangedate"]
                    data_base[day] = rate

        day = day + datetime.timedelta(days=1)
    
    return data_base

start_day = datetime.date(year = 2021, month = 1, day = 1)

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [5]:
m_list = []
for i in range (8):
    start_date = datetime.date(year = 2021, month = i+1, day = 1)
    end_date = datetime.date(year = 2021, month = i+2, day = 1)
    
    m_list.append(currency_data(start_date, end_date))

In [6]:
import math

m_str_dev = {}
m_mean = {}
i = 1

for month in m_list:
    m_mean[f'month {i}'] = sum(month.values())/len(month.values())
    m_str_dev[f'month {i}'] = math.sqrt(sum(list(map(lambda x:
                             (x - m_mean[f'month {i}'])**2, month.values())))/len(month.values()))
    i +=1
    
print("Середнє значення за кожний місяць: ", m_mean)
print("Відхилення курсу за кожний місяць: ", m_str_dev)

Середнє значення за кожний місяць:  {'month 1': 28.219770967741937, 'month 2': 27.88466071428571, 'month 3': 27.79555806451613, 'month 4': 27.930006666666667, 'month 5': 27.602848387096767, 'month 6': 27.23987333333332, 'month 7': 27.214548387096777, 'month 8': 26.768063999999995}
Відхилення курсу за кожний місяць:  {'month 1': 0.12115058663084945, 'month 2': 0.10439519125243586, 'month 3': 0.113636249822287, 'month 4': 0.08930549790217604, 'month 5': 0.11849255828264615, 'month 6': 0.1534613869639165, 'month 7': 0.15329376367376968, 'month 8': 0.10304657541131546}


4. Дану інформацію записати у файл за допомогою pickle.

In [7]:
import pickle


f = open('curr_d.pickle', 'wb')

data = {"Mean values per month":m_mean, "Standard_deviation per month" : m_str_dev}

pickle.dump(data, f)

f.close()

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [8]:
import sqlite3

db_conn = sqlite3.connect('agrosem_prod.db')

sql_text = """
            CREATE TABLE Product (
                Артикул товару TEXT,
                Найменування TEXT,
                Ціна TEXT,
                Пакування TEXT)  """

curs = db_conn.cursor()

curs.executescript(sql_text)

db_conn.close()

In [9]:
from bs4 import BeautifulSoup

conn = sqlite3.connect('agrosem_prod.db')

for page in range(8):
    url = 'https://smallpacking.agrosem.ua/shop/page'+str(page+1)+'/'
    r = requests.get(url)
    s = BeautifulSoup(r.text, 'lxml')
    prod_cont = s.find_all("div", class_ = "product-from-category-container")

    for s in prod_cont:
        r = requests.get(s.find('a').get("href"))
        s = BeautifulSoup(r.text, 'lxml')
        descr = s.find('div', class_='atributes-description')
        prod_name = descr.find('h2').text
        pack_type = descr.find('span', class_='type-packaging-name').text
        sku = descr.find('span', class_='sku').text[4:]
        price = descr.find('span', class_='regular-price').text

        text = f"('{sku}', '{prod_name}', '{price}', '{pack_type}')"
        sql_text = f"""INSERT INTO Product VALUES {text}"""

        curs = conn.cursor()
        curs.execute(sql_text)
        conn.commit()


conn.close()

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [10]:
import sqlite3

conn = sqlite3.connect('tysql.sqlite')
curs = conn.cursor()

print("Версія БД", sqlite3.sqlite_version)

Версія БД 3.35.4


2. Виведіть перелік всіх таблиць.

In [11]:
conn = sqlite3.connect('tysql.sqlite')
curs = conn.cursor()

curs.execute('SELECT name from sqlite_master where type= "table"')

print(curs.fetchall())

[('Customers',), ('OrderItems',), ('Orders',), ('Products',), ('Vendors',)]


3. Список всіх cust_id з таблиці Customers table.

In [12]:
conn = sqlite3.connect('tysql.sqlite')
curs = conn.cursor()

curs.execute('SELECT cust_id from Customers')
rows = curs.fetchall()
print(rows)

[('1000000001',), ('1000000002',), ('1000000003',), ('1000000004',), ('1000000005',)]


4. Всю таблицю Customers table

In [13]:
curs.execute('SELECT * from Customers')
rows = curs.fetchall()
print(rows)

[('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com'), ('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None), ('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com'), ('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com'), ('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)]


5. Список клієнтів (cust_names) відсортованих від Z до A

In [14]:
curs.execute('SELECT cust_name FROM Customers ORDER BY cust_name DESC')
cust = curs.fetchall()
print(cust)

[('Village Toys',), ('The Toy Store',), ('Kids Place',), ('Fun4All',), ('Fun4All',)]


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [15]:
curs.execute('SELECT cust_id, order_num FROM Orders ORDER BY cust_id, order_date')
 
row = curs.fetchall()
print(row)

[('1000000001', 20009), ('1000000001', 20005), ('1000000003', 20006), ('1000000004', 20007), ('1000000005', 20008)]


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [16]:
curs.execute('SELECT prod_id, quantity, item_price FROM OrderItems ORDER BY quantity DESC, item_price DESC')

row = curs.fetchall()
print(row)

[('BNBG01', 250, 2.49), ('BNBG02', 250, 2.49), ('BNBG03', 250, 2.49), ('BR03', 100, 10.99), ('BR01', 100, 5.49), ('BNBG01', 100, 2.99), ('BNBG02', 100, 2.99), ('BNBG03', 100, 2.99), ('BR03', 50, 11.49), ('RGAN01', 50, 4.49), ('BR01', 20, 5.99), ('BR03', 10, 11.99), ('BR02', 10, 8.99), ('BNBG01', 10, 3.49), ('BNBG02', 10, 3.49), ('BNBG03', 10, 3.49), ('BR03', 5, 11.99), ('RGAN01', 5, 4.99)]


8. Товар (з таблиці Products), ціна якого становить 9.49

In [17]:
curs.execute('SELECT prod_name, prod_id FROM Products WHERE prod_price = 9.49')

row = curs.fetchall()
print(row)

[('King doll', 'RYL01'), ('Queen doll', 'RYL02')]


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [18]:
curs.execute('SELECT prod_name, prod_price FROM Products WHERE 3 <= prod_price and prod_price <=6 ORDER BY prod_price ASC')

row = curs.fetchall()
print(row)

[('Fish bean bag toy', 3.49), ('Bird bean bag toy', 3.49), ('Rabbit bean bag toy', 3.49), ('Raggedy Ann', 4.99), ('8 inch teddy bear', 5.99)]


10. Кількість товару, що було продано

In [19]:
curs.execute('SELECT quantity FROM OrderItems ')

row = curs.fetchall()
sum_q = 0
for i in row:
    sum_q += i[0]
print(sum_q)

1430


11. Кількість найменувань товару, ціна якого більша за 4

In [20]:
curs.execute('SELECT prod_name FROM Products WHERE 4 <= prod_price')

row = curs.fetchall()
print(len(row))

6


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [21]:
curs.execute('SELECT * FROM Products ORDER BY prod_price DESC LIMIT 3')

row = curs.fetchall()
print(row)

[('BR03', 'BRS01', '18 inch teddy bear', 11.99, '18 inch teddy bear, comes with cap and jacket'), ('RYL01', 'FNG01', 'King doll', 9.49, '12 inch king doll with royal garments and crown'), ('RYL02', 'FNG01', 'Queen doll', 9.49, '12 inch queen doll with royal garments and crown')]


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [22]:
curs.execute('''SELECT cust_name, 
            (SELECT COUNT(order_num) FROM Orders WHERE Customers.cust_id = Orders.cust_id)
             FROM Customers''')
row = curs.fetchall()
print(row)

[('Village Toys', 2), ('Kids Place', 0), ('Fun4All', 1), ('Fun4All', 1), ('The Toy Store', 1)]


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [23]:
curs.execute('''SELECT cust_name, prod_name 
             FROM Customers JOIN Orders
              ON Customers.cust_id = Orders.cust_id
              JOIN OrderItems
              ON Orders.order_num = OrderItems.order_num
              JOIN Products
              ON OrderItems.prod_id = Products.prod_id''')

row = curs.fetchall()
print(row)

[('Village Toys', '8 inch teddy bear'), ('Village Toys', '18 inch teddy bear'), ('Fun4All', '8 inch teddy bear'), ('Fun4All', '12 inch teddy bear'), ('Fun4All', '18 inch teddy bear'), ('Fun4All', '18 inch teddy bear'), ('Fun4All', 'Fish bean bag toy'), ('Fun4All', 'Bird bean bag toy'), ('Fun4All', 'Rabbit bean bag toy'), ('Fun4All', 'Raggedy Ann'), ('The Toy Store', 'Raggedy Ann'), ('The Toy Store', '18 inch teddy bear'), ('The Toy Store', 'Fish bean bag toy'), ('The Toy Store', 'Bird bean bag toy'), ('The Toy Store', 'Rabbit bean bag toy'), ('Village Toys', 'Fish bean bag toy'), ('Village Toys', 'Bird bean bag toy'), ('Village Toys', 'Rabbit bean bag toy')]


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [24]:
curs.execute('''SELECT prod_name FROM Products WHERE prod_price < 5
             UNION
                SELECT prod_name FROM Products WHERE prod_price >= 5 ''')
 
row = curs.fetchall()
print(row)
conn.close()

[('12 inch teddy bear',), ('18 inch teddy bear',), ('8 inch teddy bear',), ('Bird bean bag toy',), ('Fish bean bag toy',), ('King doll',), ('Queen doll',), ('Rabbit bean bag toy',), ('Raggedy Ann',)]


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


1. SELECT model, speed, hd FROM PC WHERE price < 500
2. SELECT DISTINCT maker FROM Product WHERE type = 'Printer'
3. SELECT model, ram, screen FROM Laptop WHERE price > 1000
4. SELECT * FROM Printer WHERE color = 'y'
5. SELECT DISTINCT model, speed, hd FROM PC WHERE  (cd = '12x' OR cd = '24x') AND price < 600
6. SELECT DISTINCT maker, speed FROM Product JOIN Laptop ON Product.model = Laptop.model WHERE hd >= 10
7. SELECT DISTINCT PC.model, price FROM PC JOIN Product ON Product.model = PC.model WHERE maker = 'B'
   UNION
   SELECT DISTINCT Laptop.model, price FROM Laptop JOIN Product ON Product.model = Laptop.model WHERE maker = 'B'
   UNION
   SELECT DISTINCT Printer.model, price FROM Printer JOIN Product ON Product.model = Printer.model WHERE maker = 'B'
8. SELECT maker FROM Product WHERE type = 'PC'
   EXCEPT
   SELECT maker FROM Product WHERE type = 'Laptop'
9. SELECT DISTINCT maker FROM Product JOIN PC ON Product.model = PC.model WHERE speed >= 450
10. SELECT DISTINCT model, price FROM printer where price = (SELECT MAX(price) FROM printer)
11. SELECT AVG(speed) FROM PC
12. SELECT AVG(speed) FROM Laptop WHERE price > 1000
13. SELECT avg(PC.speed) FROM PC JOIN Product ON PC.model = Product.model WHERE maker = 'A'
14. SELECT Ships.class, Ships.name, Classes.country FROM Classes 
    JOIN Ships ON Classes.class = Ships.class WHERE numguns >= '10'
15. SELECT hd FROM PC group by hd having count(model) >= 2
16. SELECT DISTINCT A.model AS model, B.model AS model, A.speed As speed, A.ram As ram 
    FROM PC AS A, PC B WHERE A.speed = B.speed AND A.ram = B.ram AND A.model > B.model
17. SELECT DISTINCT type, model, speed FROM Laptop, (SELECT type FROM Product) AS Prod(type) 
    WHERE speed < ALL (SELECT speed FROM PC) and type = 'laptop'
18. Select distinct p.maker, pr.price from product p join printer pr on p.model = pr.model 
    WHERE pr.price = (SELECT MIN(price) FROM printer where color = 'y') and pr.color = 'y'
19. Select p.maker, AVG(l.screen) from product p join laptop l on p.model = l.model group by p.maker
20. Select maker, count(model) as Count_Model from product WHERE type = 'pc' group by maker having count(model) >= 3
21. Select p.maker, max(pc.price) as max_price from product p join pc pc on p.model = pc.model group by maker
22. Select speed, avg(price) from pc where speed > '600' group by speed
23. Select p.maker from product p join pc pc on p.model = pc.model where pc.speed >= '750' 
    intersect 
    Select p.maker from product p join laptop l on p.model = l.model where l.speed >= '750'
24. WITH all_model AS (SELECT model, price FROM pc 
    UNION 
    ALL SELECT model, price FROM printer 
    UNION 
    ALL SELECT model, price FROM laptop ) 
    SELECT distinct model FROM all_model WHERE price = ALL ( SELECT max(price) FROM all_model)
25. SELECT distinct p.maker from product p join pc on p.model = pc.model 
    WHERE pc.ram = (select min(ram) from pc) and pc.speed = (SELECT MAX(speed) FROM pc WHERE ram = (SELECT MIN(ram) 
    FROM pc)) and p.maker in (SELECT maker FROM product WHERE type = 'printer')
26. SELECT AVG(price) as Avg_price FROM (SELECT price FROM PC WHERE model IN (SELECT model FROM product WHERE maker='A' 
    AND type='PC') 
    UNION 
    ALL SELECT price FROM Laptop WHERE model IN (SELECT model FROM product WHERE maker='A' AND type='Laptop') ) AS prods
27. SELECT p.maker, avg(pc.hd) from product p join pc pc on p.model = pc.model 
    WHERE pc.model IN (SELECT model FROM pc) AND maker IN (SELECT maker FROM product WHERE type='printer') group by maker
28. SELECT count(maker) AS qty FROM (SELECT distinct maker FROM product GROUP BY maker having count(model) = 1) AS prod
29. SELECT i.point, i.date, inc, out FROM income_o i LEFT JOIN outcome_o o on i.point = o.point AND i.date = o.date
    UNION
    SELECT o.point, o.date, inc, out FROM income_o i RIGHT JOIN outcome_o o ON i.point = o.point AND i.date = o.date
30. SELECT point, date, SUM(sum_out), SUM(sum_inc) FROM( SELECT point, date, SUM(inc) as sum_inc, null as sum_out 
    FROM Income Group by point, date
    UNION
    SELECT point, date, null as sum_inc, SUM(out) as sum_out FROM Outcome Group by point, date ) as t
    GROUP BY point, date ORDER BY point